In [ ]:
# 无监督关键词提取
+ 基于特征:TF-IDF 
+ 基于词图模型:PageRank,TextRank
+ 基于主题模型:LDA

In [ ]:
# 有监督关键词提取
+ 将关键词抽取过程视为二分类问题，先提取出候选词，然后对于每个候选词划定标签，要么是关键词，要么不是关键词，然后训练关键词抽取分类器。当新来一篇文档时，提取出所有的候选词，然后利用训练好的关键词提取分类器，对各个候选词进行分类，最终将标签为关键词的候选词作为关键词

In [ ]:
# 相关工具
+ jieba
+ textrank4zh:textrank算法工具
+ snownlp
+ textblob: 英文

In [1]:
# TF-IDF
## TF-IDF 算法介绍及实现 https://blog.csdn.net/asialee_bird/article/details/81486700
## 简单结构并没有考虑词语的语义信息，无法处理一词多义与一义多词的情况
# -*- coding: utf-8 -*-
from collections import defaultdict
import math
import operator
 
"""
函数说明:创建数据样本
Returns:
    dataset - 实验样本切分的词条
    classVec - 类别标签向量
"""
def loadDataSet():
    dataset = [ ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],    # 切分的词条
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid'] ]
    classVec = [0, 1, 0, 1, 0, 1]  # 类别标签向量，1代表好，0代表不好
    return dataset, classVec
 
 
"""
函数说明：特征选择TF-IDF算法
Parameters:
     list_words:词列表
Returns:
     dict_feature_select:特征选择词字典
"""
def feature_select(list_words):
    #总词频统计
    doc_frequency=defaultdict(int)
    for word_list in list_words:
        for i in word_list:
            doc_frequency[i]+=1
 
    #计算每个词的TF值
    word_tf={}  #存储没个词的tf值
    for i in doc_frequency:
        word_tf[i]=doc_frequency[i]/sum(doc_frequency.values())
 
    #计算每个词的IDF值
    doc_num=len(list_words)
    word_idf={} #存储每个词的idf值
    word_doc=defaultdict(int) #存储包含该词的文档数
    for i in doc_frequency:
        for j in list_words:
            if i in j:
                word_doc[i]+=1
    for i in doc_frequency:
        word_idf[i]=math.log(doc_num/(word_doc[i]+1))
 
    #计算每个词的TF*IDF的值
    word_tf_idf={}
    for i in doc_frequency:
        word_tf_idf[i]=word_tf[i]*word_idf[i]
 
    # 对字典按值由大到小排序
    dict_feature_select=sorted(word_tf_idf.items(),key=operator.itemgetter(1),reverse=True)
    return dict_feature_select
 
if __name__=='__main__':
    data_list,label_list=loadDataSet() #加载数据
    features=feature_select(data_list) #所有词的TF-IDF值
    print(features)
    print(len(features))


[('to', 0.0322394037469742), ('stop', 0.0322394037469742), ('worthless', 0.0322394037469742), ('my', 0.028288263356383563), ('dog', 0.028288263356383563), ('him', 0.028288263356383563), ('stupid', 0.028288263356383563), ('has', 0.025549122992281622), ('flea', 0.025549122992281622), ('problems', 0.025549122992281622), ('help', 0.025549122992281622), ('please', 0.025549122992281622), ('maybe', 0.025549122992281622), ('not', 0.025549122992281622), ('take', 0.025549122992281622), ('park', 0.025549122992281622), ('dalmation', 0.025549122992281622), ('is', 0.025549122992281622), ('so', 0.025549122992281622), ('cute', 0.025549122992281622), ('I', 0.025549122992281622), ('love', 0.025549122992281622), ('posting', 0.025549122992281622), ('garbage', 0.025549122992281622), ('mr', 0.025549122992281622), ('licks', 0.025549122992281622), ('ate', 0.025549122992281622), ('steak', 0.025549122992281622), ('how', 0.025549122992281622), ('quit', 0.025549122992281622), ('buying', 0.025549122992281622), ('f

In [3]:
# TextRank
## https://blog.csdn.net/asialee_bird/article/details/96894533
## PageRank算法根据网页之间的链接关系构造网络，TextRank算法根据词之间的共现关系构造网络；
## PageRank算法构造的网络中的边是有向无权边，TextRank算法构造的网络中的边是无向有权边。
# coding=utf-8
from textrank4zh import TextRank4Keyword, TextRank4Sentence
import jieba.analyse
from snownlp import SnowNLP
import pandas as pd
import numpy as np
 
#关键词抽取
def keywords_extraction(text):
    tr4w = TextRank4Keyword(allow_speech_tags=['n', 'nr', 'nrfg', 'ns', 'nt', 'nz'])
    # allow_speech_tags   --词性列表，用于过滤某些词性的词
    tr4w.analyze(text=text, window=2, lower=True, vertex_source='all_filters', edge_source='no_stop_words',
                 pagerank_config={'alpha': 0.85, })
    # text    --  文本内容，字符串
    # window  --  窗口大小，int，用来构造单词之间的边。默认值为2
    # lower   --  是否将英文文本转换为小写，默认值为False
    # vertex_source  -- 选择使用words_no_filter, words_no_stop_words, words_all_filters中的哪一个来构造pagerank对应的图中的节点
    #                -- 默认值为`'all_filters'`，可选值为`'no_filter', 'no_stop_words', 'all_filters'
    # edge_source  -- 选择使用words_no_filter, words_no_stop_words, words_all_filters中的哪一个来构造pagerank对应的图中的节点之间的边
    #              -- 默认值为`'no_stop_words'`，可选值为`'no_filter', 'no_stop_words', 'all_filters'`。边的构造要结合`window`参数
 
    # pagerank_config  -- pagerank算法参数配置，阻尼系数为0.85
    keywords = tr4w.get_keywords(num=6, word_min_len=2)
    # num           --  返回关键词数量
    # word_min_len  --  词的最小长度，默认值为1
    return keywords
 
#关键短语抽取
def keyphrases_extraction(text):
    tr4w = TextRank4Keyword()
    tr4w.analyze(text=text, window=2, lower=True, vertex_source='all_filters', edge_source='no_stop_words',
                 pagerank_config={'alpha': 0.85, })
    keyphrases = tr4w.get_keyphrases(keywords_num=6, min_occur_num=1)
    # keywords_num    --  抽取的关键词数量
    # min_occur_num   --  关键短语在文中的最少出现次数
    return keyphrases
 
#关键句抽取
def keysentences_extraction(text):
    tr4s = TextRank4Sentence()
    tr4s.analyze(text, lower=True, source='all_filters')
    # text    -- 文本内容，字符串
    # lower   -- 是否将英文文本转换为小写，默认值为False
    # source  -- 选择使用words_no_filter, words_no_stop_words, words_all_filters中的哪一个来生成句子之间的相似度。
    # 		  -- 默认值为`'all_filters'`，可选值为`'no_filter', 'no_stop_words', 'all_filters'
    # sim_func -- 指定计算句子相似度的函数
 
    # 获取最重要的num个长度大于等于sentence_min_len的句子用来生成摘要
    keysentences = tr4s.get_key_sentences(num=3, sentence_min_len=6)
    return keysentences
 
 
def keywords_textrank(text):
    keywords = jieba.analyse.textrank(text, topK=6)
    return keywords
 
 
if __name__ == "__main__":
    text = "来源：中国科学报本报讯（记者肖洁）又有一位中国科学家喜获小行星命名殊荣！4月19日下午，中国科学院国家天文台在京举行“周又元星”颁授仪式，" \
           "我国天文学家、中国科学院院士周又元的弟子与后辈在欢声笑语中济济一堂。国家天文台党委书记、" \
           "副台长赵刚在致辞一开始更是送上白居易的诗句：“令公桃李满天下，何须堂前更种花。”" \
           "据介绍，这颗小行星由国家天文台施密特CCD小行星项目组于1997年9月26日发现于兴隆观测站，" \
           "获得国际永久编号第120730号。2018年9月25日，经国家天文台申报，" \
           "国际天文学联合会小天体联合会小天体命名委员会批准，国际天文学联合会《小行星通报》通知国际社会，" \
           "正式将该小行星命名为“周又元星”。"
    #关键词抽取
    keywords=keywords_extraction(text)
    print(keywords)
 
    #关键短语抽取
    keyphrases=keyphrases_extraction(text)
    print(keyphrases)
 
    #关键句抽取
    keysentences=keysentences_extraction(text)
    print(keysentences)



Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/ph/31crq1pj14n9pcl2xffj45j8t0c5l2/T/jieba.cache
Loading model cost 1.324 seconds.
Prefix dict has been built succesfully.
[{'word': '小行星', 'weight': 0.05808441467341854}, {'word': '天文台', 'weight': 0.05721653775742513}, {'word': '命名', 'weight': 0.0485177005159723}, {'word': '中国', 'weight': 0.045716478124251815}, {'word': '中国科学院', 'weight': 0.037818937836996636}, {'word': '国家', 'weight': 0.03438059254484016}]
['小行星命名']
[{'index': 4, 'sentence': '2018年9月25日，经国家天文台申报，国际天文学联合会小天体联合会小天体命名委员会批准，国际天文学联合会《小行星通报》通知国际社会，正式将该小行星命名为“周又元星”', 'weight': 0.2281040325096452}, {'index': 3, 'sentence': '”据介绍，这颗小行星由国家天文台施密特CCD小行星项目组于1997年9月26日发现于兴隆观测站，获得国际永久编号第120730号', 'weight': 0.2106246105971721}, {'index': 1, 'sentence': '4月19日下午，中国科学院国家天文台在京举行“周又元星”颁授仪式，我国天文学家、中国科学院院士周又元的弟子与后辈在欢声笑语中济济一堂', 'weight': 0.2020923401661083}]


In [4]:
# 基于jieba 的 TextRank 实现

if __name__ == "__main__":
    text = "来源：中国科学报本报讯（记者肖洁）又有一位中国科学家喜获小行星命名殊荣！4月19日下午，中国科学院国家天文台在京举行“周又元星”颁授仪式，" \
           "我国天文学家、中国科学院院士周又元的弟子与后辈在欢声笑语中济济一堂。国家天文台党委书记、" \
           "副台长赵刚在致辞一开始更是送上白居易的诗句：“令公桃李满天下，何须堂前更种花。”" \
           "据介绍，这颗小行星由国家天文台施密特CCD小行星项目组于1997年9月26日发现于兴隆观测站，" \
           "获得国际永久编号第120730号。2018年9月25日，经国家天文台申报，" \
           "国际天文学联合会小天体联合会小天体命名委员会批准，国际天文学联合会《小行星通报》通知国际社会，" \
           "正式将该小行星命名为“周又元星”。"
 
    # 基于jieba的textrank算法实现
    keywords=keywords_textrank(text)
    print(keywords)

['小行星', '命名', '国际', '中国', '国家', '天文学家']


In [5]:
# 基于SnowNLP的textrank算法实现
snlp=SnowNLP(text)
print(snlp.keywords(6))  #关键词抽取
print(snlp.summary(3))   #关键句抽取

['行星', '中国', '小', '周', '天文台', '月']
['这颗小行星由国家天文台施密特CCD小行星项目组于1997年9月26日发现于兴隆观测站', '中国科学院国家天文台在京举行“周又元星”颁授仪式', '国际天文学联合会《小行星通报》通知国际社会']


In [7]:
# LDA
#

from gensim import corpora, models
import jieba.posseg as jp
import jieba
 
 
# 简单文本处理
def get_text(text):
    flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
    stopwords = ('的', '就', '是', '用', '还', '在', '上', '作为')  # 停用词
    words_list = []
    for text in texts:
        words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords]
        words_list.append(words)
    return words_list
 
 
# 生成LDA模型
def LDA_model(words_list):
    # 构造词典
    # Dictionary()方法遍历所有的文本，为每个不重复的单词分配一个单独的整数ID，同时收集该单词出现次数以及相关的统计信息
    dictionary = corpora.Dictionary(words_list)
    print(dictionary)
    print('打印查看每个单词的id:')
    print(dictionary.token2id)  # 打印查看每个单词的id
 
    # 将dictionary转化为一个词袋
    # doc2bow()方法将dictionary转化为一个词袋。得到的结果corpus是一个向量的列表，向量的个数就是文档数。
    # 在每个文档向量中都包含一系列元组,元组的形式是（单词 ID，词频）
    corpus = [dictionary.doc2bow(words) for words in words_list]
    print('输出每个文档的向量:')
    print(corpus)  # 输出每个文档的向量
 
    # LDA主题模型
    # num_topics -- 必须，要生成的主题个数。
    # id2word    -- 必须，LdaModel类要求我们之前的dictionary把id都映射成为字符串。
    # passes     -- 可选，模型遍历语料库的次数。遍历的次数越多，模型越精确。但是对于非常大的语料库，遍历太多次会花费很长的时间。
    lda_model = models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=10)
 
    return lda_model
 
 
if __name__ == "__main__":
    texts = ['作为千元机中为数不多拥有真全面屏的手机，OPPO K3一经推出，就簇拥不少粉丝', \
             '很多人在冲着这块屏幕购买了OPPO K3之后，发现原来K3的过人之处不止是在屏幕上', \
             'OPPO K3的消费者对这部手机总体还是十分满意的', \
             '吉利博越PRO在7月3日全新吉客智能生态系统GKUI19发布会上正式亮相', \
             '今年上海车展，长安CS75 PLUS首次亮相', \
             '普通版车型采用的是双边共双出式排气布局；运动版本车型采用双边共四出的排气布局']
    # 获取分词后的文本列表
    words_list = get_text(texts)
    print('分词后的文本：')
    print(words_list)
 
    # 获取训练后的LDA模型
    lda_model = LDA_model(words_list)
 
    # 可以用 print_topic 和 print_topics 方法来查看主题
    # 打印所有主题，每个主题显示5个词
    topic_words = lda_model.print_topics(num_topics=2, num_words=5)
    print('打印所有主题，每个主题显示5个词:')
    print(topic_words)
 
    # 输出该主题的的词及其词的权重
    words_list = lda_model.show_topic(0, 5)
    print('输出该主题的的词及其词的权重:')
    print(words_list)

分词后的文本：
[['拥有', '真', '全面', '手机', 'OPPO', 'K3', '一经', '推出', '簇拥', '不少', '粉丝'], ['人', '屏幕', '购买', 'OPPO', 'K3', '发现', '原来', 'K3', '不止', '屏幕'], ['OPPO', 'K3', '消费者', '部手机', '总体'], ['吉利', '博越', 'PRO', '全新', '吉客', '智能', 'GKUI19', '发布会', '亮相'], ['上海', '长安', 'CS75', 'PLUS', '亮相'], ['版', '车型', '采用', '双边', '共', '出式', '排气', '布局', '版本', '车型', '采用', '双边', '共', '排气', '布局']]
Dictionary(42 unique tokens: ['K3', 'OPPO', '一经', '不少', '全面']...)
打印查看每个单词的id:
{'K3': 0, 'OPPO': 1, '一经': 2, '不少': 3, '全面': 4, '手机': 5, '拥有': 6, '推出': 7, '真': 8, '簇拥': 9, '粉丝': 10, '不止': 11, '人': 12, '原来': 13, '发现': 14, '屏幕': 15, '购买': 16, '总体': 17, '消费者': 18, '部手机': 19, 'GKUI19': 20, 'PRO': 21, '亮相': 22, '全新': 23, '博越': 24, '发布会': 25, '吉利': 26, '吉客': 27, '智能': 28, 'CS75': 29, 'PLUS': 30, '上海': 31, '长安': 32, '共': 33, '出式': 34, '双边': 35, '布局': 36, '排气': 37, '版': 38, '版本': 39, '车型': 40, '采用': 41}
输出每个文档的向量:
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(0, 2), (1, 1), (11, 1), (12, 1)

In [ ]:
# word2vec
1、Word2Vec词向量表示

利用浅层神经网络模型自动学习词语在语料库中的出现情况，把词语嵌入到一个高维的空间中，通常在100-500维，在高维空间中词语被表示为词向量的形式。

特征词向量的抽取是基于已经训练好的词向量模型。

2、K-means聚类算法

聚类算法旨在数据中发现数据对象之间的关系，将数据进行分组，使得组内的相似性尽可能的大，组间的相似性尽可能的小。

算法思想是：首先随机选择K个点作为初始质心，K为用户指定的所期望的簇的个数，通过计算每个点到各个质心的距离，将每个点指派到最近的质心形成K个簇，然后根据指派到簇的点重新计算每个簇的质心，重复指派和更新质心的操作，直到簇不发生变化或达到最大的迭代次数则停止。

3、基于Word2Vec词聚类关键词提取方法的实现过程

主要思路是对于用词向量表示的词语，通过K-Means算法对文章中的词进行聚类，选择聚类中心作为文本的一个主要关键词，计算其他词与聚类中心的距离即相似度，选择topK个距离聚类中心最近的词作为关键词，而这个词间相似度可用Word2Vec生成的向量计算得到。

具体步骤如下：

对语料进行Word2Vec模型训练，得到词向量文件；
对文本进行预处理获得N个候选关键词；
遍历候选关键词，从词向量文件中提取候选关键词的词向量表示；
对候选关键词进行K-Means聚类，得到各个类别的聚类中心（需要人为给定聚类的个数）；
计算各类别下，组内词语与聚类中心的距离（欧几里得距离或曼哈顿距离），按聚类大小进行降序排序；
对候选关键词计算结果得到排名前TopK个词语作为文本关键词。
 
注：第三方工具包Scikit-learn提供了K-Means聚类算法的相关函数，本文用到了sklearn.cluster.KMeans()函数执行K-Means算法，sklearn.decomposition.PCA()函数用于数据降维以便绘制图形。

In [9]:
# 信息增益
## https://blog.csdn.net/asialee_bird/article/details/81084783
from math import log

"""
函数说明:创建测试数据集
"""
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],         #数据集
               [0, 0, 0, 1, 'no'],
               [0, 1, 0, 1, 'yes'],
               [0, 1, 1, 0, 'yes'],
               [0, 0, 0, 0, 'no'],
               [1, 0, 0, 0, 'no'],
               [1, 0, 0, 1, 'no'],
               [1, 1, 1, 1, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [2, 0, 1, 2, 'yes'],
               [2, 0, 1, 1, 'yes'],
               [2, 1, 0, 1, 'yes'],
               [2, 1, 0, 2, 'yes'],
               [2, 0, 0, 0, 'no']]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']        #分类属性
    return dataSet, labels                           #返回数据集和分类属性
 
 
"""
函数说明:计算给定数据集的经验熵(香农熵)
Parameters:
    dataSet - 数据集
Returns:
    shannonEnt - 经验熵(香农熵)
"""
def calcShannonEnt(dataSet):
    numEntires = len(dataSet)                        #返回数据集的行数
    labelCounts = {}                                 #保存每个标签(Label)出现次数的字典
    for featVec in dataSet:                          #对每组特征向量进行统计
        currentLabel = featVec[-1]                   #提取标签(Label)信息
        if currentLabel not in labelCounts.keys():   #如果标签(Label)没有放入统计次数的字典,添加进去
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1               #Label计数
    shannonEnt = 0.0                                 #经验熵(香农熵)
    for key in labelCounts:                          #计算香农熵
        prob = float(labelCounts[key]) / numEntires  #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2)            #利用公式计算
    return shannonEnt                                #返回经验熵(香农熵)
 
 
"""
函数说明:按照给定特征划分数据集
Parameters:
    dataSet - 待划分的数据集
    axis - 划分数据集的特征
    value - 需要返回的特征的值
"""
def splitDataSet(dataSet, axis, value):
    retDataSet = []                                     #创建返回的数据集列表
    for featVec in dataSet:                             #遍历数据集
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]             #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])     #将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet                                   #返回划分后的数据集
 
 
"""
函数说明:选择最优特征
Parameters:
    dataSet - 数据集
Returns:
    bestFeature - 信息增益最大的(最优)特征的索引值
"""
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1                     #特征数量
    baseEntropy = calcShannonEnt(dataSet)                 #计算数据集的香农熵
    bestInfoGain = 0.0                                    #信息增益
    bestFeature = -1                                      #最优特征的索引值
    for i in range(numFeatures):                          #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        newEntropy = 0.0                                   #经验条件熵
        for value in uniqueVals:                           #计算信息增益
            subDataSet = splitDataSet(dataSet, i, value)           #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))           #计算子集的概率
            newEntropy += prob * calcShannonEnt(subDataSet)        #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy                        #信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))             #打印每个特征的信息增益
        if (infoGain > bestInfoGain):                              #计算信息增益
            bestInfoGain = infoGain                                #更新信息增益，找到最大的信息增益
            bestFeature = i                                        #记录信息增益最大的特征的索引值
    return bestFeature                                             #返回信息增益最大的特征的索引值
 
 
if __name__ == '__main__':
    dataSet, features = createDataSet()
    entropy=calcShannonEnt(dataSet)
    bestfeature=chooseBestFeatureToSplit(dataSet)
    print("训练集的熵为:%f"%(entropy))
    print("最优特征索引值:" + str(bestfeature))

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
训练集的熵为:0.970951
最优特征索引值:2


In [8]:
# 互信息
## https://blog.csdn.net/asialee_bird/article/details/96454544
## 

from sklearn import metrics
import numpy as np
 
# 训练集和训练标签
x_train = [[1, 2, 3, 4, 5],
           [5, 4, 3, 2, 1],
           [3, 3, 3, 3, 3],
           [1, 1, 1, 1, 1]]
y_train = [0, 1, 0, 1]
# 测试集和测试标签
x_test = [[2, 2, 2, 2, 2], [2, 1, 1, 2, 1]]
 
x_train = np.array(x_train)  # 转为array
 
# 存储每个特征与标签相关性得分
features_score_list = []
for i in range(len(x_train[0])):
    # 计算每个特征与标签的互信息
    feature_info = metrics.mutual_info_score(y_train, x_train[:, i])
    features_score_list.append(feature_info)

print(features_score_list)

[0.3465735902799726, 0.6931471805599452, 0.21576155433883565, 0.6931471805599452, 0.6931471805599452]


In [10]:
# 卡方
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
 
# 训练集和训练标签
x_train = [[1, 2, 3, 4, 5],
           [5, 4, 3, 2, 1],
           [3, 3, 3, 3, 3],
           [1, 1, 1, 1, 1]]
y_train = [0, 1, 0, 1]
# 测试集和测试标签
x_test = [[2, 2, 2, 2, 2], [2, 1, 1, 2, 1]]
y_test = [1, 1]
 
# 卡方检验选择特征
chi2_model = SelectKBest(chi2, k=3)  # 选择k个最佳特征
# 该函数选择训练集里的k个特征，并将训练集转化所选特征
x_train_chi2 = chi2_model.fit_transform(x_train, y_train)
# 将测试集转化为所选特征
x_test_chi2 = chi2_model.transform(x_test)
 
print('各个特征的得分：', chi2_model.scores_)
print('各个特征的p值：', chi2_model.pvalues_)  # p值越小，置信度越高，得分越高
print('所选特征的索引：', chi2_model.get_support(True))
print('特征提取转换后的训练集和测试集...')
print('x_train_chi2:', x_train_chi2)
print('x_test_chi2:', x_test_chi2)

各个特征的得分： [0.4 0.  0.4 1.6 3.6]
各个特征的p值： [0.52708926 1.         0.52708926 0.20590321 0.05777957]
所选特征的索引： [2 3 4]
特征提取转换后的训练集和测试集...
x_train_chi2: [[3 4 5]
 [3 2 1]
 [3 3 3]
 [1 1 1]]
x_test_chi2: [[2 2 2]
 [1 2 1]]


In [ ]:
# 基于树模型